In [23]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.model import CoeditorModel, _Tokenizer
from coeditor.dataset import TokenizedEditDataset
from coeditor.history import show_change
from coeditor.encoding import TokenizedEdit, decode_tokens
import shutil
import random

os.chdir(proj_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model = CoeditorModel.load_pretrained(get_model_dir() / "small")
model.to("cuda")

In [20]:
n_samples = 50

test_data: TokenizedEditDataset = pickle_load(get_dataset_dir("small") / "tokenized-file_based" / "test.pkl")
samples = [e for e in test_data.all_edits() if len(e.output_tks) < 30]
random.seed(123)
random.shuffle(samples)
samples = samples[:n_samples]

In [27]:
out_dir = Path("output/inspect_coeditor_model")
shutil.rmtree(out_dir, ignore_errors=True)
out_dir.mkdir(exist_ok=True, parents=True)

for i, ex in enumerate(tqdm(samples)):
    change = ex.as_change()
    (out_dir / f"{i}-truth.txt").write_text(show_change(change, name=str(ex.path)))
    pred_tks = model.predict(ex.input_tks)
    pred_change = TokenizedEdit(ex.path, ex.input_tks, pred_tks).as_change()
    (out_dir / f"{i}-pred.txt").write_text(show_change(pred_change, name=str(ex.path)))
    compare_str = f"* Ground truth: {decode_tokens(ex.output_tks)}\n*Prediction: {decode_tokens(pred_tks)}\n"
    (out_dir / f"{i}-compare.txt").write_text(compare_str)


100%|██████████| 50/50 [03:31<00:00,  4.24s/it]


In [26]:
print(decode_tokens(samples[1].output_tks))

<extra_id_0><extra_id_1> <add>     self.url = None
<extra_id_2><extra_id_3><extra_id_4> <add>     self.data = {}
 <del> <extra_id_5><extra_id_6><extra_id_7><extra_id_8>
